In [6]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
from sklearn import linear_model
import matplotlib.pyplot as plt

path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA'
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir(path)
file_list.sort()

In [3]:
class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
class categorise_simple():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        self.threshold.append(np.percentile(array,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)
        return np.asarray(temp)    
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(11)
                elif i<self.threshold[1][1]:
                    temp.append(12)
                elif i<self.threshold[1][2]:
                    temp.append(13)
                elif i<self.threshold[1][3]:
                    temp.append(14)
                elif i<self.threshold[1][4]:
                    temp.append(15)
                elif i<self.threshold[1][5]:
                    temp.append(16)
                elif i<self.threshold[1][6]:
                    temp.append(17)
                elif i<self.threshold[1][7]:
                    temp.append(18)
                elif i<self.threshold[1][8]:
                    temp.append(19)                    
                else:
                    temp.append(20)
            if i<0:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)
                elif i<self.threshold[0][8]:
                    temp.append(9)                    
                else:
                    temp.append(10)
        return np.asarray(temp)    
    
class categorise_x(): #flexible number of categories
    
    def __init__(self,x):
        self.threshold=[]
        self.percentiles=[]
        self.num=x
        for i in range(1,x):
            self.percentiles.append(i*100/x)        
            
    def fit(self,array):
        
        positive=array[array>0]
        negative=array[array<=0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
        
    def return_quartile(self,array):
        temp=[]
        for num in array:
            if num<0:
                counter=0
                for i in self.threshold[0]:
                    if num>=i:
                        counter+=1
                    else:
                        break
                temp.append(counter+1)
            else:
                counter=0
                for i in self.threshold[1]:
                    if num>=i:
                        counter+=1
                    else:
                        break
                temp.append(counter+self.num+1)
        return np.asarray(temp)
    
class cross():
    def __init__(self):
        self.time_last_cross=0
        self.current_sign=True
        self.last_time=datetime(1900, 1, 1, 8, 59)
    def get_time(self,time,price):
        if (time-self.last_time)>timedelta(minutes=1):
            self.last_time=time
            self.time_last_cross=time
            return 0
        self.last_time=time
        if (price>0) and self.current_sign : #if price positive and current trend is also positive
            return (time-self.time_last_cross).total_seconds()
        elif (price<0) and (not self.current_sign): #if price negative and current trend is negative
            return (time-self.time_last_cross).total_seconds()
        else: #if price positive, trend negative or price negative, trend positive
            self.time_last_cross=time
            self.current_sign=(price>0)
            return 0

In [94]:
def calc_smart_price(bid_price,bid_vol,ask_price,ask_vol):
    bid_price=bid_price.copy()
    ask_price=ask_price.copy()
    bid_vol=bid_vol.copy()
    ask_vol=ask_vol.copy()    
    
    if len(bid_price)!= len(bid_vol) or len(bid_vol)!=len(ask_price) or len(ask_price)!=len(ask_vol) or len(ask_vol)!=len(bid_price):
        return
    
    #to combat the limit up event, where price is set to 0. 
    rows=(bid_price==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        bid_price[rows]=ask_price[rows] #for that row, assign ask price to it
    rows=(ask_price==0)
 #do the same for ask price
    if (np.any(rows)):
        ask_price[rows]=bid_price[rows]
    smart_price=(bid_price*ask_vol+ask_price*bid_vol)/(ask_vol+bid_vol)    
    return smart_price

def calc_present_vol(vol):
    vol=vol.copy()
    vol=np.diff(vol)
    vol=np.insert(vol,0,0)
    return vol

def calc_future_price(time_array, price_array,time_ahead=30):
    if len(time_array)!=len(price_array):
        return
    time_array=time_array.copy()
    price_array=price_array.copy()
    future_price=[]
    length=len(time_array)
    for i in range(len(time_array)):
        current_time=time_array[i]+timedelta(seconds=time_ahead)
        
        j=0 #could alternatively use 30 x 3 then search forward and backward
        
        #search forwards
        while((i+j)<length and current_time>time_array[i+j]):
            j+=1
        if (i+j)<length:
            #if index is in the dataframe
            future_price.append(price_array[(i+j)]) 
        else:
            #price ahead does not exist
            future_price.append(np.nan) 
    future_price=np.asarray(future_price)
    return future_price

def calc_edge(future_price,current_price):
    future_price=future_price.copy()
    current_price=current_price.copy()
    return (future_price-current_price)

def set_index(time_array):
    time_array=time_array.copy().astype(int)
    new_index=[]
    for j in range(len(time_array)):
        i=str(time_array[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    new_index=np.asarray(new_index)
    return new_index

def calc_sma_fast(price_array,time_array,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    price_array=price_array.copy()
    time_array=time_array.copy()
    if len(price_array)!=len(time_array):
        return
    sma_values=[] 
    smart_sum=np.cumsum(price_array) 
    for i in range(len(price_array)):
        
        #finding ending point
        last_time=time_array[i]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if time_array[i-j]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and time_array[i-j]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                sma=(smart_sum[i]-smart_sum[i-j])/j
                sma_values.append(sma)                
                
            else: 
                
                #search forward
                while(time_array[i-j]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if time_array[i-j]!=last_time:
                        j+=1
                    sma=(smart_sum[i]-smart_sum[i-j])/j
                    sma_values.append(sma)   
                    
                else:
                    sma_values.append(price_array[i])
                    
        else: #starting point is at 0
            
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)                       

    sma_values=np.asarray(sma_values)
    return sma_values

def ewma_array(array,alpha=0.05,confidence=0.99,tolerance=1e-07):
    final=0
    array=alpha*array
    
    if len(array)==1:
        return(array[0])
    
    #array needs to have at least n samples to achieve confidence
    n=np.log((1-confidence))/np.log((1-alpha))
    
    for i in range(1,len(array)+1):
        current_day_ewma=array[-i]*((1-alpha)**(i-1))
        
        if (current_day_ewma<tolerance):
            #print('break')
            break
        final+=current_day_ewma 
        
    if len(array)<=n:
        final=final/(1-(1-alpha)**len(array))
        #print('adjusted',n)
    return final 

In [92]:
x=np.array([1,2,3,4,5,6,7,8,7,6,5,4,3,2])
print(len(x))
#x=np.concatenate((x,x,x,x,x,x,x,x,x,x))
ewma_array(x,alpha=0.1)

14


4.529082013227519

In [93]:
def calc_ewma(price_array,time_array,duration=1,alpha=0.1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    price_array=price_array.copy()
    time_array=time_array.copy()
    
    if len(price_array)!=len(time_array):
        return
    if len(price_array)==1:
        return price_array
    
    ewma_values=[] 
    
    for i in range(len(price_array)):
        
        #finding ending point
        last_time=time_array[i]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if time_array[i-j]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and time_array[i-j]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                ewma=ewma_array(price_array[i-j:i+1])
                ewma_values.append(ewma)       
                
            else: 
                
                #search forward
                while(time_array[i-j]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if time_array[i-j]!=last_time:
                        j+=1
                    ewma=ewma_array(price_array[i-j:i+1])
                    ewma_values.append(ewma)   
                    
                else:
                    ewma_values.append(price_array[i])
                    
        else: #starting point is at 0
            
            ewma=ewma_array(price_array[:i+1])
            ewma_values.append(ewma)                

    ewma_values=np.asarray(ewma_values)
    return ewma_values


def calc_past_vol(vol_array,time_array,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    vol_array=vol_array.copy()
    time_array=time_array.copy()
    if len(vol_array)!=len(time_array):
        return
    vol_values=[] 
    vol_sum=np.cumsum(vol_array) 
    for i in range(len(price_array)):
        
        #finding ending point
        last_time=vol_array[i]-timedelta(minutes=duration)
        
        #finding start point
        j=220*duration#4x60=240
        
        if i-j>0:
            if time_array[i-j]>last_time: 
                
                #if starting point time is greater than ending point time
                #search backward
                while(i-j>0 and time_array[i-j]>last_time):
                    j+=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('backward',i,j,data[i,time_index],data[i-j,time_index],last_time)

                vol=(vol_sum[i]-vol_sum[i-j])
                vol_values.append(vol)                
                
            else: 
                
                #search forward
                while(time_array[i-j]<last_time):
                    j-=1
                    
                #activate next line in order to debug and troubleshoot
                #if data[i-j,time_index]!=last_time:
                #    print('forward',i,j,data[i,time_index],data[i-j,time_index],last_time)
                
                if j!=0:
                    if time_array[i-j]!=last_time:
                        j+=1
                    vol=(smart_sum[i]-smart_sum[i-j])
                    vol_values.append(vol)   
                    
                else: #if j=0, means i-j=i and past volume is 0
                    vol_values.append(0)
                    
        else: #starting point is at 0
            
            vol=vol_sum[i]
            vol_values.append(vol)                       

    vol_values=np.asarray(vol_values)
    return vol_values

def last_cross(dataset,price_array,time_array):
    price_array=price_array.copy()
    time_array=time_array.copy()
    last_cross=cross()
    timings=[]
    for i in range(len(price-array)):
        timings.append(last_cross.get_time(time_array[i],price_array[i]))
    timings=np.asarray(timings)
    return timings

def get_case(array):
    current=1
    previous=0
    temp=[]
    for i in array:
        if i==current:
            if i>previous:
                temp.append(1)
            else:
                temp.append(0)
        elif i>current:
            previous=current
            current=i
            temp.append(1)
        else:
            previous=current
            current=i
            temp.append(0)
    return np.asarray(temp)


def binaryToDecimal(binary): 
      
    binary1 = binary 
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return(decimal) 
    
def get_case_n(array,n=2):
    temp=[]
    if n==1:
        return("error")
    previous=np.zeros(n+1)
    for i in array:
        #print(i,previous)
        final=''
        if i==int(previous[-1]):
            before=10
            for j in previous:
                if j>before:
                    final+='1'
                else:
                    final+='0'
                before=int(j)
            final=final[1:]
        else:
            before=10
            for j in previous[1:]:
                if j>before:
                    final+='1'
                else:
                    final+='0'
                before=int(j)
            final=final[1:]
            if i>before:
                final+='1'
            else:
                final+='0'
        
            previous=np.delete(previous,0)
            previous=np.insert(previous,n,i)
        #print(final)
        temp.append(binaryToDecimal(int(final)))
    return np.asarray(temp)    
    

def process(dataset,sma_duration=1,**args):
    data=dataset.copy().loc[:,['Time','BidPrice1','AskPrice1','BidVol1','AskVol1']].values
    
    #calc smart price 
    smart_price=calc_smart_price(data[:,1],data[:,3],data[:,2],data[:,4])
    
    #set index to datetime format
    time_array=set_index(data[:,0]) 
    
    #calculate future price
    future_price=calc_future_price(time_array,smart_price)
    
    #calculate edge 
    edge=calc_edge(future_price=future_price,current_price=smart_price)
    
    #calculate SMA
    sma=calc_sma_fast(time_array=time_array,price_array=smart_price,duration=1)
    
    smart_price=np.expand_dims(smart_price,axis=1)
    time_array=np.expand_dims(time_array,axis=1)
    future_price=np.expand_dims(future_price,axis=1)
    edge=np.expand_dims(edge,axis=1)
    sma=np.expand_dims(sma,axis=1)
    
    data=np.concatenate((time_array,smart_price,future_price,edge,sma),axis=1)
    
    return data

def process_macd(dataset,sma_duration=1):
    data=dataset.copy().loc[:,['Time','BidPrice1','AskPrice1','BidVol1','AskVol1']].values
    
    #calc smart price 
    smart_price=calc_smart_price(data[:,1],data[:,3],data[:,2],data[:,4])
    
    #set index to datetime format
    time_array=set_index(data[:,0]) 
    
    #calculate future price
    future_price=calc_future_price(time_array,smart_price)
    
    #calculate edge 
    edge=calc_edge(future_price=future_price,current_price=smart_price)
    
    #calculate SMA
    ewma_1=calc_ewma(time_array=time_array,price_array=smart_price,duration=1)
    ewma_16=calc_ewma(time_array=time_array,price_array=smart_price,duration=16)    
    macd=ewma_1-ewma_16
    
    smart_price=np.expand_dims(smart_price,axis=1)
    time_array=np.expand_dims(time_array,axis=1)
    future_price=np.expand_dims(future_price,axis=1)
    edge=np.expand_dims(edge,axis=1)
    ewma=np.expand_dims(ewma,axis=1)
    
    data=np.concatenate((time_array,smart_price,future_price,edge,ewma),axis=1)
    
    return data

In [52]:
#demonstration of get_case_n function
x=get_case_n(np.array([12,14,13,4,4,4]),2)
x

array([1, 3, 2, 0, 0, 0])

In [70]:
#processing raw data to get technicals
df_list=[]
name_list=[]
ma_time=10
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
for file in file_list: #read all files and add them to file_list
    if file[-3:]=='csv': #check if file is a CSV
        name_list.append(file)
        df_list.append(process_light(pd.read_csv(path+file),sma_duration=ma_time))
        print(file,'read')

2019.01.02.csv read
2019.01.03.csv read
2019.01.04.csv read
2019.01.07.csv read
2019.01.08.csv read
2019.01.09.csv read
2019.01.10.csv read
2019.01.11.csv read
2019.01.14.csv read
2019.01.15.csv read
2019.01.16.csv read
2019.01.17.csv read
2019.01.18.csv read
2019.01.21.csv read
2019.01.22.csv read
2019.01.23.csv read
2019.01.24.csv read
2019.01.25.csv read
2019.01.28.csv read
2019.01.29.csv read
2019.01.30.csv read
2019.01.31.csv read
2019.02.01.csv read
2019.02.11.csv read
2019.02.12.csv read
2019.02.13.csv read
2019.02.14.csv read
2019.02.15.csv read
2019.02.18.csv read
2019.02.19.csv read
2019.02.20.csv read
2019.02.21.csv read
2019.02.22.csv read
2019.02.25.csv read
2019.02.26.csv read
2019.02.27.csv read
2019.02.28.csv read
2019.03.01.csv read
2019.03.04.csv read
2019.03.05.csv read
2019.03.06.csv read
2019.03.07.csv read
2019.03.08.csv read
2019.03.11.csv read
2019.03.12.csv read
2019.03.13.csv read
2019.03.14.csv read
2019.03.15.csv read
2019.03.18.csv read
2019.03.19.csv read


In [71]:
#calculating results
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'
analysis_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/case analysis/'
n=2

#create a list to hold all the data
data_list=[]
#creating a list for each category
for i in range(20):
    data_list.append([])
    #creating a list for each quartile
    for _ in range((2**n)):
        data_list[i].append([])
        
for i in range(len(df_list)): #for each 20 day rolling window
    if i<19: #skip first 19 days
        continue
    print(name_list[i])

    #get -19 day
    sma=df_list[i-19][:,-1].copy() #column for SMA
    
    #get -18 to 0 day (19 days in total)
    for k in range((i-18),i+1): #get 20 day moving averages
        sma=np.concatenate((sma,df_list[k][:,-1].copy()))
        
    cat_sma=categorise_10()
    cat_sma.fit(sma) #calculate quartile thresholds for past 20 days

    #get x,y for regression
    x_today=df_list[i][:,-1].copy().astype(float) #column for SMA     
    y_today=df_list[i][:,-2].copy().astype(float) #column for edge
    
    #removing all NA
    isnum=(~np.isnan(x_today)) & (~np.isnan(y_today))
    x_today=x_today[isnum]
    y_today=y_today[isnum]
    
    #get categories of today's sma  
    cat_x_today=cat_sma.return_quartile(x_today)

    change=get_case_n(cat_x_today.copy())

    #for each category
    for cat in range(1,21):
    
        #today's sma filter
        sma_filter_today=(cat_x_today==cat)

        #for each case
        for case in range(2**n):
                
            filtered= sma_filter_today & (change==case) #filtering by SMA and case
            filtered[:2]=False #removing first 2 entries due to insufficient data
            new_y=y_today[filtered].copy()

            if (len(new_y)!=0):
                #add today's data into the list by category and case
                data_list[(cat-1)][case].append(new_y) 
                
final_df=pd.DataFrame()                 
for cat in range(20):
    for case in range(2**n):
        reg_result={}
        if len(data_list[cat][case])==0:
            reg_result['category']=cat+1
            reg_result['case']=case+1
            reg_result['mean']=np.nan
            reg_result['std']=np.nan
            reg_result['num obs']=0
            final_df=final_df.append(reg_result,ignore_index=True)             
            continue
            
        #getting all the data needed to calculate mean and std
        all_data=data_list[cat][case][0].copy()
        for i in range(1,(len(data_list[cat][case]))):
            all_data=np.concatenate((all_data,data_list[cat][case][i].copy()))

        #adding results to output
        reg_result['category']=cat+1
        reg_result['case']=case+1
        reg_result['mean']=np.mean(all_data)
        reg_result['std']=np.std(all_data)
        reg_result['num obs']=len(all_data)
        final_df=final_df.append(reg_result,ignore_index=True) 
             
temp=df_path+'result_10split_'+str(2**n)+'cases_'+str(ma_time)+'min_sma_weightedmean_new.csv'
final_df.to_csv(temp)
print('done',temp)

2019.01.29.csv
2019.01.30.csv
2019.01.31.csv
2019.02.01.csv
2019.02.11.csv
2019.02.12.csv
2019.02.13.csv
2019.02.14.csv
2019.02.15.csv
2019.02.18.csv
2019.02.19.csv
2019.02.20.csv
2019.02.21.csv
2019.02.22.csv
2019.02.25.csv
2019.02.26.csv
2019.02.27.csv
2019.02.28.csv
2019.03.01.csv
2019.03.04.csv
2019.03.05.csv
2019.03.06.csv
2019.03.07.csv
2019.03.08.csv
2019.03.11.csv
2019.03.12.csv
2019.03.13.csv
2019.03.14.csv
2019.03.15.csv
2019.03.18.csv
2019.03.19.csv
2019.03.20.csv
2019.03.21.csv
2019.03.22.csv
2019.03.25.csv
2019.03.26.csv
2019.03.27.csv
2019.03.28.csv
2019.03.29.csv
2019.04.01.csv
2019.04.02.csv
2019.04.03.csv
2019.04.04.csv
2019.04.08.csv
2019.04.09.csv
2019.04.10.csv
2019.04.11.csv
2019.04.12.csv
2019.04.15.csv
2019.04.16.csv
2019.04.17.csv
2019.04.18.csv
2019.04.19.csv
2019.04.22.csv
2019.04.23.csv
2019.04.24.csv
2019.04.25.csv
2019.04.26.csv
2019.04.29.csv
2019.04.30.csv
2019.05.06.csv
2019.05.07.csv
2019.05.08.csv
2019.05.09.csv
done /Users/hudsonyeo/Desktop/Python/leo

In [57]:
#interpretation of case results
for x in ['0','1']:
    for y in ['0','1']:
        print(x+y,binaryToDecimal(int(x+y))+1)
# 0 indicates decrease, 1 indicates increase

00 1
01 2
10 3
11 4


In [72]:
for time in [1,3,5,10]:
    try:
        data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_10split_4cases_'+str(time)+'min_sma_weightedmean.csv')
    except:
        continue
    length=len(data)
    print(time,'minute MA')
    
    print('mean')
    print(np.reshape(data.loc[:,'mean'].values,(20,4)))
    print('std')
    print(np.reshape(data.loc[:,'std'].values,(20,4)))
    print('num obs')
    x1=np.reshape(data.loc[:,'num obs'].values,(20,4))
    print(np.reshape(data.loc[:,'num obs'].values,(20,4)))

1 minute MA
mean
[[-0.10123765         nan -0.22224872         nan]
 [-0.21845741  0.08265356 -0.34498651         nan]
 [-0.27631761 -0.03164807 -0.33938919  0.03531408]
 [-0.20701956 -0.02084277 -0.16618014  0.13997762]
 [-0.30108895  0.01080844 -0.32577298  0.18572127]
 [-0.29864848 -0.0557268  -0.0973267   0.16684549]
 [-0.26975039 -0.04590963 -0.13803736  0.16018784]
 [-0.36018879 -0.03437015 -0.19268968  0.17345851]
 [-0.28373105 -0.02128981 -0.09507237  0.20451147]
 [-0.32085268  0.05248896 -0.14408516  0.19682824]
 [-0.2294909   0.16387443 -0.18530666  0.26140565]
 [-0.22123366  0.09640732 -0.08697581  0.26075438]
 [-0.21236527  0.24242495 -0.02913809  0.30804359]
 [-0.19267353  0.21498476 -0.05540559  0.27182402]
 [-0.14719781  0.16353381 -0.00784121  0.27025168]
 [-0.09709343  0.40686389  0.04342834  0.25684396]
 [-0.12037499  0.34109759  0.05604844  0.28712703]
 [-0.10929727  0.31791287  0.04905949  0.23291513]
 [        nan  0.4190952  -0.0443465   0.33643283]
 [        nan 

In [73]:
for time in [1,3,5,10]:
    try:
        data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_10split_4cases_'+str(time)+'min_sma_weightedmean_new.csv')
    except:
        continue
    length=len(data)
    print(time,'minute MA')
    print('mean')
    print(np.reshape(data.loc[:,'mean'].values,(20,4)))
    print('std')
    print(np.reshape(data.loc[:,'std'].values,(20,4)))
    print('num obs')
    x2=np.reshape(data.loc[:,'num obs'].values,(20,4))
    print(np.reshape(data.loc[:,'num obs'].values,(20,4)))

1 minute MA
mean
[[-0.15707977         nan -0.01526818         nan]
 [-0.22432963  0.07451811 -0.20702137         nan]
 [-0.30849692 -0.04560965 -0.26019859  0.03381705]
 [-0.23318078 -0.03591182 -0.16739341  0.01558353]
 [-0.36404783 -0.06298882 -0.21863177  0.13420138]
 [-0.31194633 -0.16356036 -0.20740974  0.09426032]
 [-0.24774466 -0.11757976 -0.26148695  0.07402522]
 [-0.35090885 -0.08119375 -0.29217667  0.02089411]
 [-0.32942903 -0.11335671 -0.15644405  0.09220461]
 [-0.25260451  0.01362005 -0.06159319  0.12512678]
 [-0.28616861  0.09718981 -0.07304333  0.23568003]
 [-0.22138117  0.20536967  0.02850372  0.23722997]
 [-0.14262826  0.25508298  0.07546102  0.32018908]
 [-0.17582056  0.2274647   0.05877015  0.28398263]
 [-0.12684284  0.21805602  0.07982552  0.26889813]
 [-0.10926458  0.25725371  0.13004272  0.29267219]
 [-0.11259938  0.27108598  0.12970074  0.32259374]
 [-0.01765611  0.2140748   0.05207068  0.27531622]
 [        nan  0.36635876 -0.04497216  0.32020018]
 [        nan 

In [ ]:
    
'''    final_df=pd.DataFrame()
    final_df['sma']=x_today
    final_df['category']=cat_x_today
    final_df['case']=change
    final_df['edge']=y_today
    temp=analysis_path+'analysis'+str(name_list[i])+str(2**n)+'cases_'+str(ma_time)+'min_sma.csv'
    final_df.to_csv(temp)'''